# 手眼标定

同济子豪兄 2024-5-13

## 导入工具包

In [1]:
from pymycobot.mycobot import MyCobot
from pymycobot import PI_PORT, PI_BAUD
import time

import cv2
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## 连接机械臂

In [2]:
mc = MyCobot(PI_PORT, PI_BAUD)

## 设置运动模式为插补

In [3]:
mc.set_fresh_mode(0)

## 机械臂归零

In [4]:
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

## 第一步：移动至俯视姿态

俯视姿态一（关节）：[0, 0, -88, 0, 0, 45]

俯视姿态二（坐标）：[13, -160, 212, 180, 3.31, -135.81]

俯视姿态二（关节）：[-62.13, 8.96, -87.71, -14.41, 2.54, -16.34]

In [5]:
mc.send_coords([13, -160, 212, 180, 3.31, -135.81], 10)
time.sleep(3)

In [6]:
mc.get_angles()

[-62.13, 8.26, -86.57, -14.32, 2.81, -16.52]

In [5]:
mc.send_angles([-62.13, 8.96, -87.71, -14.41, 2.54, -16.34], 10)
time.sleep(3)

## 第二步：在白纸靠左下角的位置，画一个标定点

## 第三步：运行check_camera.py，打开摄像头实时画面

## 第四步：把白纸右上角对准画面右上角

## 第五步：白纸上边与底座、图像上边平齐，白纸下边与图像下边平齐

## 第六步：用夹子固定白纸，分别夹左上角和右下角。（把麦克风线也固定）

## 第七步：通过鼠标点选，获取白纸左下角标定点，在图像上的像素坐标

In [6]:
# 第一个标定点的像素坐标
cali_1_im = [130, 290]

## 第八步：控制机械臂，移动至左下角第一个标定点

In [7]:
# 移动到标定点附近
mc.send_coords([-36, -205, 94.7, 178.92, 4.56, -135.57], 10)
time.sleep(3)

In [8]:
mc.get_coords()

[-34.1, -205.0, 92.8, 178.45, 4.98, -135.44]

In [9]:
X = -36

In [13]:
X += 15
mc.send_coord(1, X, 20)
time.sleep(1)

In [14]:
Y = -205

In [15]:
Y -= 8
mc.send_coord(2, Y, 20)
time.sleep(1)

In [7]:
Z = 95

In [8]:
Z -= 10
mc.send_coord(3, Z, 20)
time.sleep(1)

In [28]:
mc.get_coords()

[-21.8, -197.4, 59.3, 177.67, 5.2, -136.02]

- 记下机械臂坐标

In [29]:
# 第一个标定点的机械臂坐标
cali_1_mc = [-21.8, -197.4]

## 第九步：控制机械臂，移动至右上角第二个标定点

In [30]:
# 第二个标定点的像素坐标
cali_2_im = [640, 0]

In [31]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

In [33]:
# 移动到第二个标定点附近
mc.send_coords([211, -62, 120, 180, 3.31, -135.81], 10)
time.sleep(3)

In [245]:
mc.get_coords()

[188.4, -49.8, 141.4, 178.41, 1.11, -135.72]

In [52]:
Z = 120

In [70]:
Z -= 15
mc.send_coord(3, Z, 20)
time.sleep(1)

In [54]:
mc.get_coords()

[211.3, -60.8, 92.8, 178.55, 2.69, -136.32]

In [56]:
X = 211

In [57]:
X += 5
mc.send_coord(1, X, 20)
time.sleep(1)

In [58]:
Y = -62

In [62]:
Y += 5 
mc.send_coord(2, Y, 20)
time.sleep(1)

In [71]:
mc.get_coords()

[215.1, -59.1, 75.8, 178.6, 2.24, -136.5]

In [72]:
# 第二个标定点的机械臂坐标
cali_2_mc = [215, -59.1]

## 第十步：通过插值，获取图像任意像素坐标对应的机械臂坐标

In [73]:
# 整理两个标定点的坐标
cali_1_im = [130, 290]                       # 左下角，第一个标定点的像素坐标，要手动填！
cali_1_mc = [-21.8, -197.4]                  # 左下角，第一个标定点的机械臂坐标，要手动填！
cali_2_im = [640, 0]                         # 右上角，第二个标定点的像素坐标
cali_2_mc = [215, -59.1]                    # 右上角，第二个标定点的机械臂坐标，要手动填！

In [74]:
# 指定点在图像中的像素坐标
X_im = 320
Y_im = 240

In [75]:
X_cali_im = [cali_1_im[0], cali_2_im[0]]     # 像素坐标
X_cali_mc = [cali_1_mc[0], cali_2_mc[0]]     # 机械臂坐标

X_mc = int(np.interp(X_im, X_cali_im, X_cali_mc))

Y_cali_im = [cali_2_im[1], cali_1_im[1]]     # 像素坐标，先小后大
Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]     # 机械臂坐标，先大后小

Y_mc = int(np.interp(Y_im, Y_cali_im, Y_cali_mc))

In [76]:
X_mc

66

In [77]:
Y_mc

-173

## 让机械臂移动至该点吸取

In [78]:
mc.send_coords([X_mc, Y_mc, 200, -178.24, 1.68, -134.33], 20)
time.sleep(3)

In [86]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

## 封装手眼标定函数

In [79]:
import numpy as np
def eye2hand(X_im=160, Y_im=120):
    '''
    输入目标点在图像中的像素坐标，转换为机械臂坐标
    '''

    # 整理两个标定点的坐标
    cali_1_im = [130, 290]                       # 左下角，第一个标定点的像素坐标，要手动填！
    cali_1_mc = [-21.8, -197.4]                  # 左下角，第一个标定点的机械臂坐标，要手动填！
    cali_2_im = [640, 0]                         # 右上角，第二个标定点的像素坐标
    cali_2_mc = [215, -59.1]                    # 右上角，第二个标定点的机械臂坐标，要手动填！
    
    X_cali_im = [cali_1_im[0], cali_2_im[0]]     # 像素坐标
    X_cali_mc = [cali_1_mc[0], cali_2_mc[0]]     # 机械臂坐标
    Y_cali_im = [cali_2_im[1], cali_1_im[1]]     # 像素坐标，先小后大
    Y_cali_mc = [cali_2_mc[1], cali_1_mc[1]]     # 机械臂坐标，先大后小

    # X差值
    X_mc = int(np.interp(X_im, X_cali_im, X_cali_mc))

    # Y差值
    Y_mc = int(np.interp(Y_im, Y_cali_im, Y_cali_mc))

    return X_mc, Y_mc

In [80]:
eye2hand(X_im=640, Y_im=0)

(215, -59)

In [81]:
eye2hand(X_im=320, Y_im=240)

(66, -173)

## 第十一步：验证标定效果

In [82]:
# 机械臂归零
mc.send_angles([0, 0, 0, 0, 0, 0], 40)
time.sleep(3)

In [83]:
# 移动至俯视姿态
mc.send_angles([-62.13, 8.96, -87.71, -14.41, 2.54, -16.34], 10)
time.sleep(3)

In [84]:
# 运行`camera_check.py`，用鼠标点选图像中的某个点，获取像素坐标
X_im, Y_im = 320, 240

In [85]:
# 手眼标定转换为机械臂坐标
X_mc, Y_mc = eye2hand(X_im, Y_im)

In [86]:
# 控制机械臂移动到这个点，看是否准确
mc.send_coords([X_mc, Y_mc, 100, -178.24, 1.68, -134.33], 20)
time.sleep(3)